In [1]:
import syllables 
import pandas as pd
import random
import numpy as np
import re
import language_tool_python
from IPython.display import display

In [2]:
with open("Dataset_poems.txt", "r",encoding="utf8") as file:
    # Trial_data.txt
    # Dataset_poems.txt
    temp = []
    for line in file:
        line1 = re.sub('[^a-zA-Z ]', '', line)
        line1 = line1.split()
        if line1 != ['', ''] and line1 != ['']:
            for i in line1:
                if i == '':
                    continue
                else:
                    temp.append(i)
words_df = temp 

In [6]:
class poem_generation: 
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    def select_random_words(self,words_df):
        words_array1 = []
        while len(words_array1) <= 15:
            random_number = random.randint(0, len(words_df)-1)
            if words_df[random_number] != 0:
                words_array1.append(words_df[random_number])
        # print(len(words_array1))
        return words_array1

    def create_poems(self,arr):
        
        poem = [0 for _ in range(4)]
        for i in range(1,5):
            temp =  arr[4*(i-1):4*i]
            poem[i-1] = temp
            poem[i-1]=' '.join(poem[i-1])
            print(poem[i-1])
        print("=========================================================")
        return poem

    def store_syllables(self,arr):
        a = [0 for _ in range(4)]
        temp = 0
        j = 0
        for words,i in zip(arr,range(0,16)):
            temp = temp + syllables.estimate(words)
            if i in [3,7,11,15]:
                a[j] = temp
                # print(a[j])
                temp = 0
                j += 1
        return a

    def Grammar_checker(self,text):
        error = 0
        poem_error = []
        tool = language_tool_python.LanguageTool('en-US')
        for lines in text:
            matches = tool.check(lines)
            if len(matches)>0:
                error = error + matches[0].errorLength
        poem_error.append(error)
        return poem_error

    def crossover(self,p1,p2,cross):
        part1 = []
        part2 = []
        part3 = []
        child1 = []
        child2 = []
        j = 0
        k = 0
        random_number1 = sorted([random.randint(1, 15) for i in range(cross)])
        random_number2 = sorted([random.randint(1, 15) for i in range(cross)])
        for i in random_number1:
            part1.append(p1[j:i])
            j = i
        for i in random_number2:
            part2.append(p2[k:i])
            k = i
        part1.append(p1[j:])
        part2.append(p2[k:])
        part2.extend(part1)
        random.shuffle(part2)
        for i in range(len(part2)):
            part3.extend(part2[i])
        for i in range(int(len(part3)/2)):
            child1.append(part3[i])
        for i in range(int(len(part3)/2),int(len(part3))):
            child2.append(part3[i])
        if child1 == p1 or child1 == p2 or child2 == p1 or child2 == p2:
            child1, child2,child1_poem,child2_poem = self.crossover(child1,child2,cross)
        child1_poem = self.create_poems(child1)
        child2_poem = self.create_poems(child2)
        return child1,child2,child1_poem,child2_poem

    def tournament(self,Population,m):
        p = int(m/3)
        n = m-p
        df = pd.DataFrame()
        # print(Population.head())
        selected_polulation = pd.DataFrame()
        Population.sort_values(by=['GrammarError'], inplace=True)
        selected_polulation = Population.iloc[:n]
        # selected_polulation = Population.loc[(Population["Syllables1"] == 5)&(Population["Syllables2"] == 6)&(Population["Syllables3"] == 6)&(Population["Syllables4"] == 5)]
        # selected_polulation = selected_polulation.iloc[:n]
        random_number = [random.sample(n, len(Population)-1) for i in range(p)]
        for i in random_number:
            df.append(Population.iloc[i])
        selected_polulation = pd.concat([selected_polulation,df],axis=0)
        print()
        print("SELECTED")
        display(selected_polulation)
        return selected_polulation,selected_polulation["words_array"]   
    def create_set_of_poems(self,n):
            poem_population = []
            syllables_count = []
            grammar_error = []
            words_array_population = []
            for i in range(n):
                words_array = self.select_random_words(words_df)
                poem = self.create_poems(words_array)
                sum_syllables = self.store_syllables(words_array)
                poem_error = self.Grammar_checker(poem)
                poem_population.append(poem)
                syllables_count.append(sum_syllables)
                grammar_error.append(poem_error)
                words_array_population.append(words_array)
            words_array2 = pd.DataFrame(dict(zip(["words_array"],[words_array_population])))
            Final = np.hstack((poem_population,syllables_count,grammar_error))
            Population_data = pd.DataFrame(Final,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError"])
            Pop_data = pd.concat([Population_data,words_array2],axis=1)
            print()
            print("PARENTS")
            display(Pop_data)
            return words_array_population,Pop_data

    def child_poems(self,words_array,cross):
        child_population = []
        syllables_count = []
        grammer_error = []
        words_array_child = []
        for i in range(0,len(words_array)-1,2):
            child1,child2,child1_poem,child2_poem = self.crossover(words_array[i],words_array[i+1],cross)
            child_population.append(child1_poem)
            child_population.append(child2_poem)
            sum_syllables1 = self.store_syllables(child1)
            sum_syllables2 = self.store_syllables(child2)
            poem_error1 = self.Grammar_checker(child1_poem)
            poem_error2 = self.Grammar_checker(child2_poem)
            syllables_count.append(sum_syllables1)
            syllables_count.append(sum_syllables2)
            grammer_error.append(poem_error1)
            grammer_error.append(poem_error2)
            words_array_child.append(child1)
            words_array_child.append(child2)
        words_array_child = pd.DataFrame(dict(zip(["words_array"],[words_array_child])))
        Final1 = np.hstack((child_population,syllables_count,grammer_error))
        child_data = pd.DataFrame(Final1,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError"])
        child_data = pd.concat([child_data,words_array_child],axis=1)
        print()
        print("CHILD")
        display(child_data)
        return words_array_child,child_data

    
    def __init__(self,words_df,n,cross,Selection_number):
        assert n>1 ,f"Population {n} cannot be less than or equal to 1"
        assert n>=Selection_number, f"Cannot select {Selection_number} poems from total pololation of {n}"
        assert cross>1, f"Minimum number for crossover should be 1"
        words_array,Population_data = self.create_set_of_poems(n)
        Selected_Population_data,selected_polulation_word_array = self.tournament(Population_data,Selection_number)
        words_array_child,child_population = self.child_poems(selected_polulation_word_array,cross)




In [7]:
#(self,words_df,n,cross,Selection_number):
poem = poem_generation(words_df,2,2,2)

Heavensupon Might to Ore
not Is home of
She the that fierce
me Swift of springs
How That my more
feat notion Air eye
it thee Sing rock
In the flies hunting
PARENTS


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
0,Heavensupon Might to Ore,not Is home of,She the that fierce,me Swift of springs,7,4,5,4,23,"[Heavensupon, Might, to, Ore, not, Is, home, of, She, the, that, fierce, me, Swift, of, springs]"
1,How That my more,feat notion Air eye,it thee Sing rock,In the flies hunting,4,5,4,5,6,"[How, That, my, more, feat, notion, Air, eye, it, thee, Sing, rock, In, the, flies, hunting]"


SELECTED


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
0,Heavensupon Might to Ore,not Is home of,She the that fierce,me Swift of springs,7,4,5,4,23,"[Heavensupon, Might, to, Ore, not, Is, home, of, She, the, that, fierce, me, Swift, of, springs]"
1,How That my more,feat notion Air eye,it thee Sing rock,In the flies hunting,4,5,4,5,6,"[How, That, my, more, feat, notion, Air, eye, it, thee, Sing, rock, In, the, flies, hunting]"


my more feat notion
Air eye it thee
Sing rock In the
flies hunting Heavensupon Might
to Ore not Is
home of She the
that fierce me Swift
of springs How That
CHILD


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
0,my more feat notion,Air eye it thee,Sing rock In the,flies hunting Heavensupon Might,5,4,4,8,7,"[my, more, feat, notion, Air, eye, it, thee, Sing, rock, In, the, flies, hunting, Heavensupon, Might]"
1,to Ore not Is,home of She the,that fierce me Swift,of springs How That,4,4,5,4,12,"[to, Ore, not, Is, home, of, She, the, that, fierce, me, Swift, of, springs, How, That]"
